### Set the variables for base model, dataset, and new model name

In [ ]:
%%capture
!pip install -r requirements.txt

In [ ]:
checkpoint_loc = "output/checkpoints"
lora_adapter_loc = "output/lora_adapter"
#dataset_name = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"

In [ ]:
%env CLEARML_LOG_MODEL=True
from clearml import Task, OutputModel, Dataset

project_name = 'userxx/CustomerSupport'
task_name = "03-fine-tune-llama3-2-1b"
s3_base_bucket_loc = 's3://tk-aip/clearml'
base_model = "/mnt/shared/models/huggingface/Llama-3.2-1B-Instruct"

task = Task.init(project_name=project_name, task_name=task_name, output_uri=s3_base_bucket_loc)
#Task.add_requirements("./requirements.txt")
#task.set_parameter("base-model",base_model)
#task.set_parameter("lora-adapter",task_name)
#task.set_base_docker(docker_image="nvidia/cuda:11.8.0-cudnn8-runtime-ubuntu22.04")
#task.set_base_docker(docker_image="nvidia/cuda:12.4.1-cudnn-runtime-ubuntu22.04")
#task.execute_remotely(queue_name="q-group-a-gpu-10gb")

### Load the Python packages and functions we will use throughout the fine-tuning and evaluation process.

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

wandb.init(mode="disabled")

## 2. Loading the model and tokenizer

### Set the data type and attention implementation

In [ ]:
# Set torch dtype and attention implementation
#if torch.cuda.get_device_capability()[0] >= 8:
#    print("install flash-attn!!!")
##    !pip install -qqq flash-attn
##    !pip install flash-attn --no-build-isolation
#    torch_dtype = torch.bfloat16
#    attn_implementation = "flash_attention_2"
#else:
#    torch_dtype = torch.float16
#    attn_implementation = "eager"

torch_dtype = torch.float16
attn_implementation = "eager"

### Load the model and tokenizer by providing the local model directory, we will load the model in 4-bit quantization.

In [ ]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)
# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

## 3. Loading and processing the dataset

In [ ]:
cml_dataset_project = project_name
cml_dataset_name = "bitext-customer-support"
cml_dataset = Dataset.get(
        dataset_project=cml_dataset_project,
        dataset_name=cml_dataset_name,
        only_completed=True,
        only_published=False,
)
dataset_path = cml_dataset.get_local_copy()
files = cml_dataset.list_files()
data_file = dataset_path + "/" + files[0]

In [ ]:
#Importing the dataset
#dataset = load_dataset(dataset_name, split="train")
dataset = load_dataset("csv", data_files=data_file, split="train")
dataset = dataset.shuffle(seed=65).select(range(1000)) # Only use 1000 samples for quick demo
instruction = """You are a top-rated customer service agent named John. 
    Be polite to customers and answer all their questions.
    """
def format_chat_template(row):
    
    row_json = [{"role": "system", "content": instruction },
               {"role": "user", "content": row["instruction"]},
               {"role": "assistant", "content": row["response"]}]
    
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc= 4,
)

In [ ]:
dataset['text'][3]
dataset = dataset.train_test_split(test_size=0.1)

## 4. Setting up the model

### Extract the linear model name from the model

In [ ]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)
#modules = ['gate_proj', 'v_proj', 'up_proj', 'q_proj', 'down_proj', 'k_proj', 'o_proj']

### Use the linear module name to create the LoRA adopter. We will only fine-tune the LoRA adopter and leave the rest of the model to save memory and for faster training time

In [ ]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)
#model, tokenizer = setup_chat_format(model, tokenizer)
tokenizer.pad_token = tokenizer.eos_token

model = get_peft_model(model, peft_config)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir=checkpoint_loc,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
#    report_to="wandb"
)

### We will now set up a supervised fine-tuning (SFT) trainer and provide a train and evaluation dataset, LoRA configuration, training argument, tokenizer, and model. 

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
#    max_seq_length= 512,
#    dataset_text_field="text",
#    tokenizer=tokenizer,
    args=training_arguments,
#    packing= False,
)

## 5. Model training

In [ ]:
trainer.train()

## 6. Model Inference

### To test the fine-tuned model, we will provide it with the sample prompt from the dataset

In [ ]:
messages = [{"role": "system", "content": instruction},
    {"role": "user", "content": "I bought the same item twice, cancel order {{Order Number}}"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")
outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text.split("assistant")[1])

## 7. Saving the tokenizer and model

### Output LoRA Adapter to local folder

In [ ]:
model.save_pretrained(lora_adapter_loc)

### Upload LoRA Adapter in Zipped format to S3 Bucket via ClearML

In [ ]:
from clearml import Task, OutputModel, StorageManager

s3_bucket_loc = s3_base_bucket_loc + "/" + Task.current_task().get_project_name() + "/" + task_name + "." + task.id + "/models/lora_adapter"
StorageManager.upload_folder(lora_adapter_loc, s3_bucket_loc)
clearmlModel = OutputModel(
    task=Task.current_task(),
    framework="PyTorch",
    name="remote_lora_adapter_loc"
)
clearmlModel.update_weights(
    register_uri=s3_bucket_loc,
    target_filename = "remote_lora_adapter_loc"
)

In [ ]:
#lora_adapter = task_name

#from clearml import Task, OutputModel
# Upload the merged model to S3 bucket
clearmlModel = OutputModel(
    task=Task.current_task(),
    framework="PyTorch",
    name="remote_lora_adapter_zipped"
)
clearmlModel.update_weights_package(
    weights_path=lora_adapter_loc,
    target_filename="remote_lora_adapter_zipped",
    auto_delete_file=False
)

In [ ]:
task.close()